In [1]:
%%time

! pip install huggify-data

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 47.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 72.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 75.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.3/328.3 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 87.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 57.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 114.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 39.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━

CPU times: user 568 ms, sys: 99.6 ms, total: 668 ms
Wall time: 1min 35s


In [1]:
from datasets import load_dataset

ds = load_dataset("Aiwensile2/StorySeed")

Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/4376 [00:00<?, ? examples/s]

In [2]:
ds

DatasetDict({
    train: Dataset({
        features: ['prompt', 'response'],
        num_rows: 4376
    })
})

In [3]:
ds['train']['prompt'][0]

'Please help me create a picture book with the theme of\nA curious kitten embarks on a whimsical journey through a magical forest, encountering friendly creatures along the way.\nThe format should include a title, followed by a cover page with an illustration only, and the remaining pages should each have both text and illustrations.'

In [4]:
ds['train']['response'][0]

'### Title: Whisker\'s Magical Forest Adventure\n\n---\n\n### Cover Page Illustration:\nA small, fluffy, gray kitten with wide, curious eyes standing at the edge of a lush, vibrant forest filled with colorful flowers and twinkling lights.\n\n---\n\n### Page 1 Illustration:\nWhiskers peeking out from behind a large oak tree, gazing at the path ahead.\n\n#### Text:\nIn the heart of a sun-dappled garden, little Whiskers the kitten found a hidden path leading into a mysterious forest. With a twitch of his nose and a flick of his tail, he decided it was the perfect day for an adventure.\n\n---\n\n### Page 2 Illustration:\nWhiskers meeting a cheerful squirrel with a bushy tail, sitting on a tree branch.\n\n#### Text:\nNot far into the forest, Whiskers met Squeaky the squirrel. "Where are you off to, little kitten?" Squeaky chirped. "I\'m exploring this magical forest," Whiskers replied with a purr. Squeaky nodded, "Follow me! I\'ll show you the way to the Whispering Willows."\n\n---\n\n### P

```python
### Human: What does YSA stand for? ### Assistant: YSA stands for Youth Spirit Artworks.
```

In [5]:
from tqdm import tqdm

In [6]:
questions = []
answers = []
texts = []

for i in tqdm(range(100)):
    ques = ds['train']['prompt'][i]
    answ = ' '.join(ds['train']['response'][i].split('### '))
    text = "### Human: " + ques + " ### Assistant: " + answ

    questions.append(ques)
    answers.append(answ)
    texts.append(text)

100%|██████████| 100/100 [00:02<00:00, 44.47it/s]


In [7]:
import pandas as pd

In [8]:
df = pd.DataFrame({'questions': questions, 'answers': answers, 'text': texts})
df.head()

,questions,answers,text
0,Please help me create a picture book with the ...,Title: Whisker's Magical Forest Adventure\n\n...,### Human: Please help me create a picture boo...
1,Please help me create a picture book with the ...,Title: **Mia and Mico’s Treasure Quest**\n\n ...,### Human: Please help me create a picture boo...
2,Please help me create a picture book with the ...,Title: The Enchanted Doorway\n\n---\n\n# Cove...,### Human: Please help me create a picture boo...
3,Please help me create a picture book with the ...,"Title: ""The Cave of Dragon Friends""\n\n---\n\...",### Human: Please help me create a picture boo...
4,Please help me create a picture book with the ...,Title: Pip's Quest for the Golden Cheese\n\n ...,### Human: Please help me create a picture boo...


In [9]:
df.to_csv('data.csv', index=False)

In [3]:
from google.colab import userdata

In [11]:
from huggify_data.push_modules import DataFrameUploader

# Example usage:
df = pd.read_csv('/content/data.csv')
uploader = DataFrameUploader(
    df,
    hf_token=userdata.get('HF_TOKEN'),
    repo_name='storySeed-v3', # this is whatever you want
    username='eagle0504' # your own user name
)
uploader.process_data()
uploader.push_to_hub()

Dataframe verified: columns 'questions' and 'answers' are present.
Data processed into DatasetDict.
Repository created: eagle0504/storySeed-v3


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Dataset pushed to Hugging Face Hub.


In [4]:
from huggify_data.train_modules import *

In [5]:
# Usage
model_name = "NousResearch/Llama-2-7b-chat-hf"
dataset_name = "eagle0504/storySeed-v3"
new_model = "storySeed-v3-llama2"
huggingface_token = userdata.get('HF_TOKEN')

In [ ]:
%%time

trainer = LlamaTrainer(model_name, dataset_name, new_model, huggingface_token)
peft_config = trainer.configure_lora()
training_args = trainer.configure_training_arguments(num_train_epochs=1)
trainer.train_model(training_args, peft_config)

In [6]:
from huggify_data.train_modules import *
from google.colab import userdata

In [9]:
%%time

# Usage
model_name = "NousResearch/Llama-2-7b-chat-hf"
dataset_name = "eagle0504/storySeed-v3"
new_model = "storySeed-v3-llama2"
huggingface_token = userdata.get('HF_TOKEN')
trainer = LlamaTrainer(model_name, dataset_name, new_model, huggingface_token)

Starting model loading...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Model loading completed.
CPU times: user 5.11 s, sys: 1.04 s, total: 6.15 s
Wall time: 7.33 s


In [11]:
some_model, some_tokenizer = trainer.load_model_and_tokenizer(
    base_model_path="NousResearch/Llama-2-7b-chat-hf",
    new_model_path="eagle0504/storySeed-v3-llama2",
)

adapter_config.json:   0%|          | 0.00/453 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_model.bin:   0%|          | 0.00/134M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/434 [00:00<?, ?B/s]

In [12]:
# Example usage
prompt = "### Human: Please help me create a picture book with the theme of A curious kitten embarks on a whimsical journey through a magical forest, encountering friendly creatures along the way. The format should include a title, followed by a cover page with an illustration only, and the remaining pages should each have both text and illustrations. ### Assistant:"
response = trainer.generate_response(some_model, some_tokenizer, prompt)
print(response)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


### Human: Please help me create a picture book with the theme of A curious kitten embarks on a whimsical journey through a magical forest, encountering friendly creatures along the way. The format should include a title, followed by a cover page with an illustration only, and the remaining pages should each have both text and illustrations. ### Assistant: Of course! Here's an outline for your picture book: Title: "Whiskers in the Woods
